In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Path ke file
file_path = '/content/drive/MyDrive/Dataset dibimbing/online_retail.csv'

# Membaca file dengan delimiter tab (\t)
df_online_retail = pd.read_csv(file_path,  encoding='ISO-8859-1', sep=';')

# Menampilkan beberapa baris pertama
df_online_retail.head()


MessageError: Error: credential propagation was unsuccessful

In [ ]:
df_online_retail.info()

#Data Cleaning and Manipulation


In [ ]:
import pandas as pd #as_Library yang digunakan untuk manipulasi dan analisis data dalam bentuk tabel atau struktur data seperti DataFrame dan Series.
import matplotlib.pyplot as plt #as_Submodul dari matplotlib yang digunakan untuk membuat grafik visualisasi data.
import numpy as np #as_Library untuk operasi numerik, seperti array multidimensi, perhitungan matematis, dan fungsi-fungsi aljabar linear.
import seaborn as sns #Library untuk visualisasi data yang lebih canggih dan menarik dibandingkan matplotlib.
from sklearn.model_selection import train_test_split #as_Fungsi dari library sklearn yang digunakan untuk membagi dataset menjadi dua bagian: data latih (training data) dan data uji (testing data). Biasanya digunakan dalam machine learning untuk mengevaluasi performa model.

# Scalling untuk MinMax Scaller
from sklearn.preprocessing import MinMaxScaler #as:Alat preprocessing dari sklearn yang digunakan untuk melakukan normalisasi data, yaitu mengubah nilai-nilai fitur menjadi skala tertentu, biasanya antara 0 dan 1.
# Set the maximum number of columns and rows to display to a large number
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# A. Handling Duplicated Data for ML

In [ ]:
#as:digunakan untuk membagi dataset df_online_retail menjadi data training dan data testing menggunakan fungsi train_test_split dari scikit-learn.
from sklearn.model_selection import train_test_split
train_online_retail, test_online_retail = train_test_split(df_online_retail, test_size = 0.2, random_state=42)#as: 0.2 itu artinya data test 20%. 42 ini biasanya yang dipakai utk randomnya.  biar sama terus datanya.

In [ ]:
train_online_retail.head()

In [ ]:
test_online_retail.head()

In [ ]:
train_online_retail.shape

ada 433.527 baris dan 9 kolom untuk data train

In [ ]:
test_online_retail.shape

ada 108.382 baris dan 9 kolom untuk data test

In [ ]:
len(train_online_retail.drop_duplicates()) / len(train_online_retail)

In [ ]:
len(test_online_retail.drop_duplicates()) / len(test_online_retail)

Tidak terdapat duplikat untuk data train ataupun test

# B. Outlier Handling For Machine Learning : InterQuartile Handling in Data Train only

Untuk variable Quantity dan UnitPrice

In [ ]:
# untuk plot Q-Q
import scipy.stats as stats

In [ ]:
def check_plot(df, variable):

    plt.figure(figsize=(16, 4))

    # histogram
    plt.subplot(1, 3, 1)
    sns.histplot(df[variable], bins=30)
    plt.title('Histogram')

    # plot Q-Q
    plt.subplot(1, 3, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    plt.ylabel('Variable quantiles')

    # box plot
    plt.subplot(1, 3, 3) #as: ini maknanya 1 baris artinya nanti grafik nya di output ada di 1 baris, angka kedua ialah banyak kolom yaitu 3 berarti nanti ada 3 grafik atau 3 kolom disebutnya, angka ketiga adalah urutan. ini no 3 berarti boxplot
    sns.boxplot(y=df[variable])
    plt.title('Boxplot')

    plt.show()

##Check for Quantity Variable

In [ ]:
check_plot(train_online_retail, 'Quantity')

In [ ]:
def find_outlier_boundary(df, variable):

    # Mari kita hitung batas luar yang merupakan outlier

    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25) #as ini rumus iqr. yaitu q1 -q3

    lower_boundary = df[variable].quantile(0.25) - (IQR * 1.5)
    upper_boundary = df[variable].quantile(0.75) + (IQR * 1.5)

    return upper_boundary, lower_boundary

In [ ]:
# batas untuk AveOccup

occup_upper_limit, occup_lower_limit = find_outlier_boundary(train_online_retail, 'Quantity')
occup_upper_limit, occup_lower_limit #as_digunakan untuk menampilkan kedua nilai tersebut.

#as: hasilnya ini maksudnya batas atas dan batas bawah. jadi batas atas =4,5 dan batas bawah=1,15

In [ ]:
train_online_retail = train_online_retail[(train_online_retail['Quantity'] <= occup_upper_limit) & (train_online_retail['Quantity'] >= occup_lower_limit)]


In [ ]:
check_plot(train_online_retail, 'Quantity')

##Check for UnitPrice Variable

In [ ]:
check_plot(train_online_retail, 'UnitPrice')

In [ ]:
occup_upper_limit, occup_lower_limit = find_outlier_boundary(train_online_retail, 'UnitPrice')
occup_upper_limit, occup_lower_limit

In [ ]:
train_online_retail = train_online_retail[(train_online_retail['UnitPrice'] <= occup_upper_limit) & (train_online_retail['UnitPrice'] >= occup_lower_limit)]


In [ ]:
check_plot(train_online_retail, 'UnitPrice')

#C. Missing Value Handling For Machine Learning

In [ ]:
df_online_retail.isna().sum()

In [ ]:
df_online_retail = df_online_retail.dropna(subset=['Description'])

In [ ]:
df_online_retail.info()

In [ ]:
df_online_retail = df_online_retail.dropna(subset=['CustomerID'])

In [ ]:
df_online_retail.info()

In [ ]:
# Membuat kolom baru 'TotalPembelian' berdasarkan perhitungan Quantity * UnitPrice
df_online_retail['TotalPembelian'] = df_online_retail['Quantity'] * df_online_retail['UnitPrice']
print(df_online_retail.head())

In [ ]:
# Mengubah tipe data kolom 'InvoiceDate' dari object ke datetime
df_online_retail['InvoiceDate'] = pd.to_datetime(df_online_retail['InvoiceDate'])

# Menampilkan DataFrame untuk memastikan perubahan tipe data
print(df_online_retail.dtypes)

#EDA

In [ ]:
df_online_retail.select_dtypes(include=['number']).describe()

In [ ]:
# Agregasi total pembelian per pelanggan
total_per_customer = df_online_retail.groupby('CustomerID')['TotalPembelian'].sum().reset_index()
print(total_per_customer.head())


In [ ]:
# Visualisasi distribusi total pembelian per pelanggan
plt.figure(figsize=(10,6))
sns.histplot(total_per_customer['TotalPembelian'], kde=True, bins=50, color='skyblue')

# Menambahkan label dan judul
plt.title('Distribusi Total Pembelian per Pelanggan', fontsize=15)
plt.xlabel('Total Pembelian', fontsize=12)
plt.ylabel('Frekuensi', fontsize=12)

# Menampilkan grafik
plt.show()

##Kategorik Total Purchase 4 kat


In [ ]:
# Menentukan interval dan label kategori
bins = [-168469, 4.2, 11.09, 19.49, 168469.6]
labels = ['Kategori 1', 'Kategori 2', 'Kategori 3', 'Kategori 4']

# Membuat kolom kategori berdasarkan TotalPurchase
df_online_retail['Kategori'] = pd.cut(df_online_retail['TotalPembelian'], bins=bins, labels=labels, right=True)

# Menampilkan beberapa data untuk memastikan kategorinya sudah sesuai
print(df_online_retail[['CustomerID', 'TotalPembelian', 'Kategori']].head())


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Membuat histogram kategori menggunakan seaborn
plt.figure(figsize=(8, 6))
ax = sns.countplot(x='Kategori', data=df_online_retail, palette='Blues')

# Menambahkan label dan judul
plt.title('Distribusi Pelanggan Berdasarkan Kuartil TotalPurchase')
plt.xlabel('Kategori')
plt.ylabel('Jumlah Pelanggan')

# Menambahkan angka di atas bar
for p in ax.patches:
    ax.annotate(f'{p.get_height()}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center',
                fontsize=12, color='black',
                xytext=(0, 5), textcoords='offset points')

# Menampilkan grafik
plt.show()


In [ ]:
# Menghitung frekuensi pembelian per pelanggan
purchase_frequency = df_online_retail.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()

# Menamai kolom baru sebagai 'Frekuensi_Pembelian'
purchase_frequency.rename(columns={'InvoiceNo': 'Frekuensi_Pembelian'}, inplace=True)

# Menampilkan hasil
print(purchase_frequency.head())


In [ ]:
purchase_frequency[['Frekuensi_Pembelian']].describe()

In [ ]:
# Visualisasi distribusi frekuensi pembelian per pelanggan
plt.figure(figsize=(10,6))
sns.histplot(purchase_frequency['Frekuensi_Pembelian'], kde=True, bins=30, color='lightcoral')

# Menambahkan label dan judul
plt.title('Distribusi Frekuensi Pembelian per Pelanggan', fontsize=15)
plt.xlabel('Frekuensi Pembelian', fontsize=12)
plt.ylabel('Jumlah Pelanggan', fontsize=12)

# Menampilkan grafik
plt.show()


##4 kategorik frekuensi pembelian

In [ ]:
# Menentukan interval dan label kategori untuk frekuensi pembelian
bins_frekuensi = [0, 1.01, 3, 5, 248]
labels_frekuensi = ['Kategori 1', 'Kategori 2', 'Kategori 3', 'Kategori 4']

# Membuat kolom kategori berdasarkan Frekuensi_Pembelian
purchase_frequency['Kategori_Frekuensi'] = pd.cut(purchase_frequency['Frekuensi_Pembelian'],
                                                   bins=bins_frekuensi,
                                                   labels=labels_frekuensi,
                                                   right=True)

# Menampilkan beberapa data untuk memastikan kategorinya sudah sesuai
print(purchase_frequency[['CustomerID', 'Frekuensi_Pembelian', 'Kategori_Frekuensi']].head())


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Membuat grafik kategori frekuensi pembelian menggunakan seaborn
plt.figure(figsize=(8, 6))
ax = sns.countplot(x='Kategori_Frekuensi', data=purchase_frequency, palette='Blues')

# Menambahkan label dan judul
plt.title('Distribusi Pelanggan Berdasarkan Kategori Frekuensi Pembelian')
plt.xlabel('Kategori Frekuensi Pembelian')
plt.ylabel('Jumlah Pelanggan')

# Menambahkan angka di atas bar
for p in ax.patches:
    ax.annotate(f'{p.get_height()}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center',
                fontsize=12, color='black',
                xytext=(0, 5), textcoords='offset points')

# Menampilkan grafik
plt.show()


In [ ]:
# Menghitung total pembelian per negara
total_per_country = df_online_retail.groupby('Country')['TotalPembelian'].sum().reset_index()

# Menampilkan hasil
print(total_per_country.head())


In [ ]:
# Visualisasi distribusi total pembelian per negara
plt.figure(figsize=(12,8))
sns.barplot(x='TotalPembelian', y='Country', data=total_per_country, palette='viridis')

# Menambahkan label dan judul
plt.title('Distribusi Total Pembelian Berdasarkan Negara', fontsize=15)
plt.xlabel('Total Pembelian', fontsize=12)
plt.ylabel('Negara', fontsize=12)

# Menampilkan grafik
plt.show()


In [ ]:
# Pastikan kolom 'InvoiceDate' dalam format datetime
df_online_retail['InvoiceDate'] = pd.to_datetime(df_online_retail['InvoiceDate'])

# Membuat kolom 'Bulan_Tahun' yang berisi bulan dan tahun
df_online_retail['Bulan_Tahun'] = df_online_retail['InvoiceDate'].dt.to_period('M')

# Menampilkan beberapa baris untuk memastikan kolom baru
print(df_online_retail[['InvoiceDate', 'Bulan_Tahun']].head())


In [ ]:
# Menghitung jumlah (count) dan persentase (percentage)
Bulan_Tahun_count = df_online_retail['Bulan_Tahun'].value_counts()
Bulan_Tahun_percentage = df_online_retail['Bulan_Tahun'].value_counts(normalize=True) * 100


# Menggabungkan dalam satu dataframe
Bulan_Tahun_summary = pd.DataFrame({'Count': Bulan_Tahun_count, 'Percentage': Bulan_Tahun_percentage})


# Menampilkan hasil
print("Ringkasan variabel 'Bulan_Tahun':\n", Bulan_Tahun_summary)

In [ ]:
print(df_online_retail.dtypes)

In [ ]:
# Menghitung total pembelian per bulan
total_per_month = df_online_retail.groupby('Bulan_Tahun')['TotalPembelian'].sum().reset_index()

# Menampilkan hasil
print(total_per_month.head())


In [ ]:
# Mengubah kolom 'Bulan_Tahun' menjadi format datetime
df_online_retail['Bulan_Tahun'] = df_online_retail['Bulan_Tahun'].dt.to_timestamp()

# Menghitung total pembelian per bulan
total_per_month = df_online_retail.groupby('Bulan_Tahun')['TotalPembelian'].sum().reset_index()

# Visualisasi tren pembelian per bulan
plt.figure(figsize=(12,6))
sns.lineplot(x='Bulan_Tahun', y='TotalPembelian', data=total_per_month, marker='o', color='teal')

# Menambahkan label dan judul
plt.title('Tren Pembelian per Bulan', fontsize=15)
plt.xlabel('Bulan dan Tahun', fontsize=12)
plt.ylabel('Total Pembelian', fontsize=12)
plt.xticks(rotation=45)  # Agar label bulan lebih mudah dibaca

# Menampilkan grafik
plt.show()


In [ ]:
# Menghitung total pembelian per produk
top_products = df_online_retail.groupby('StockCode')['Quantity'].sum().reset_index()

# Mengurutkan produk berdasarkan total pembelian (dari yang paling banyak)
top_products_sorted = top_products.sort_values(by='Quantity', ascending=False)

# Menampilkan 10 produk teratas yang paling banyak dibeli
print(top_products_sorted.head(10))


In [ ]:
# Visualisasi top 10 produk yang paling banyak dibeli
plt.figure(figsize=(12,6))
sns.barplot(x='Quantity', y='StockCode', data=top_products_sorted.head(10), palette='Blues_d')

# Menambahkan label dan judul
plt.title('Top 10 Produk yang Paling Banyak Dibeli', fontsize=15)
plt.xlabel('Jumlah Pembelian', fontsize=12)
plt.ylabel('Kode Produk (StockCode)', fontsize=12)

# Menampilkan grafik
plt.show()


#Feature Engineering

##1. membuat kolom rfm


Perlu menghitung tiga fitur utama, yaitu Recency, Frequency, dan Monetary.

Langkah-langkah membuat kolom RFM:
Recency: Menghitung selisih hari antara tanggal pembelian terakhir dan tanggal hari ini.
Frequency: Menghitung jumlah transaksi yang dilakukan oleh pelanggan.
Monetary: Menghitung total pembelian (nilai Quantity * UnitPrice dalam dataset).

In [ ]:
# Menghitung total pembelian per transaksi
df_online_retail['TotalPurchase'] = df_online_retail['Quantity'] * df_online_retail['UnitPrice']

# Recency: Menghitung waktu sejak pembelian terakhir per pelanggan
most_recent_date = df_online_retail['InvoiceDate'].max()
recency = df_online_retail.groupby('CustomerID')['InvoiceDate'].max().reset_index()
recency['Recency'] = (most_recent_date - recency['InvoiceDate']).dt.days

# Frequency: Menghitung jumlah transaksi per pelanggan
frequency = df_online_retail.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()
frequency.rename(columns={'InvoiceNo': 'Frequency'}, inplace=True)

# Monetary: Menghitung total pembelian per pelanggan
monetary = df_online_retail.groupby('CustomerID')['TotalPurchase'].sum().reset_index()
monetary.rename(columns={'TotalPurchase': 'Monetary'}, inplace=True)

# Menggabungkan fitur-fitur RFM
rfm_df = recency.merge(frequency, on='CustomerID').merge(monetary, on='CustomerID')

# Menyimpan datatset RFM
rfm_df.to_csv('RFM_features.csv', index=False)


Preprocessing dan Normalisasi Data

##Normalisasi fitur

Perlu menormalisasi fitur RFM agar fitur dengan rentang yang lebih besar tidak mendominasi perhitungan jarak. Normalisasi menggunakan StandardScaler.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Menggunakan RFM features untuk normalisasi
rfm_scaled = rfm_df[['Recency', 'Frequency', 'Monetary']]

# Melakukan standardisasi pada data
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_scaled)

# Menyimpan data yang sudah dinormalisasi
rfm_scaled_df = pd.DataFrame(rfm_scaled, columns=['Recency', 'Frequency', 'Monetary'])


#Modeling dengan KNN (K-Nearest Neighbors)

##a. Menentukan jumlah cluster dengan Elbow Method

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Tentukan jumlah cluster yang ingin dicoba
inertia = []
for k in range(1, 11):  # Mengecek dari 1 hingga 10 cluster
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(rfm_scaled_df)
    inertia.append(kmeans.inertia_)

# Visualisasi Elbow Method
plt.plot(range(1, 11), inertia, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.show()


Berdasarkan grafik, K=3 atau K=4 adalah jumlah klaster yang optimal untuk model K-Means, yang dapat digunakan untuk membagi data menjadi klaster-klaster yang lebih baik dan lebih informatif.

##b. Latih Model KMeans untuk Segmentasi Pelanggan

Fungsi melatih model KMeans untuk segmentasi pelanggan adalah untuk mengelompokkan pelanggan yang memiliki karakteristik perilaku yang serupa berdasarkan data yang dimiliki.

In [ ]:
# Tentukan jumlah cluster optimal (misalnya, 4 berdasarkan Elbow Method)
kmeans = KMeans(n_clusters=4, random_state=42)

# Melatih model dengan data yang sudah dinormalisasi
rfm_df['Cluster'] = kmeans.fit_predict(rfm_scaled_df)

# Menyimpan hasil clustering
rfm_df.to_csv('Customer_Segmentation_Results.csv', index=False)

#Evaluasi Model KNN dengan Silhouette Score

In [ ]:
from sklearn.metrics import silhouette_score

# Menghitung Silhouette Score
sil_score = silhouette_score(rfm_scaled_df, rfm_df['Cluster'])
print(f"Silhouette Score: {sil_score}")


Nilai Silhouette Score sebesar 0.614 menunjukkan bahwa hasil clustering atau klasifikasi relatif baik

#Analisis Hasil Cluster

In [ ]:
# Menghitung rata-rata RFM untuk setiap cluster
cluster_analysis = rfm_df.groupby('Cluster').agg({'Recency': 'mean', 'Frequency': 'mean', 'Monetary': 'mean'}).reset_index()

# Visualisasi cluster
import seaborn as sns
sns.pairplot(rfm_df, hue='Cluster', vars=['Recency', 'Frequency', 'Monetary'])
plt.show()


Hubungan antara Recency, Frequency, dan Monetary di setiap klaster. Distribusi menunjukkan bahwa masing-masing klaster memiliki karakteristik unik.


In [ ]:
#Download data yang sudah final

In [ ]:
# Menyimpan DataFrame ke file CSV
df_online_retail.to_csv('df_online_retail.csv', index=False)